In [1]:
! pip install langchain

In [2]:
! pip install langchain_community langchain-google-genai python-dotenv streamlit langchain_experimental sentence-transformers langchain_chroma langchainhub pypdf rapidocr-onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.3 MB/s eta 0:00

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("THE CONSTITUTION OF INDIA.pdf")
data = loader.load()  # entire PDF is loaded as a single Document
#data

In [6]:
len(data)

88

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2250)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  165


In [8]:
docs[7]

Document(metadata={'source': 'THE CONSTITUTION OF INDIA.pdf', 'page': 5}, page_content='6                             THE CONSTITUTION OF INDIA \n \nPART III  \nFUNDAMENTAL RIGHTS  \nGeneral  \n12. Definition. —In this Part, unless the context otherwise requires, “the \nState’’ includes the Government and Parliament of India and the \nGovernment and the Legislature of each of the States and all local or other \nauthorities within the territory of India or under the control of the \nGovernment of India. \n13. Laws inconsistent with or in derogation of the fundamental \nrights. —(1) All laws in force in the territory of India immediately before \nthe commencement of this Constitution, in so far as they are inconsistent \nwith the provisions of this Part, shall, to the extent of such inconsistency, \nbe void. \n(2) The State shall not make any law which takes away or abridges the \nrights conferred by this Part and any law made in contravention of this \nclause shall, to the extent of the

In [9]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

#Get an API key:
# Head to https://ai.google.dev/gemini-api/docs/api-key to generate a Google AI API key. Paste in .env file

# Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]
#vector

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [10]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [11]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("What is the Indian Constitution?")

In [12]:
len(retrieved_docs)

10

In [13]:
print(retrieved_docs[5].page_content)

45                             THE CONSTITUTION OF INDIA 
 
that period resolutions disapproving it are passed by both Houses, upon the passing of the 
second of those resolutions; and 
(b) may be withdrawn at any time by the President. 
Explanation. —Where the Houses of Parliament are summoned to 
reassemble on different dates, the period of six weeks shall be reckoned 
from the later of those dates for the purposes of this clause. 
(3) If and so far as an Ordinance under this article makes any provision 
which Parliament would not under this Constitution be competent to enact, 
it shall be void. 
*           *               *              *                   *     
CHAPTER IV.—T HE UNION JUDICIARY  
124. Establishment and constitution of Supreme Court.— (1) There 
shall be a Supreme Court of India consisting of a Chief Justice of India 
and, until Parliament by law prescribes a larger number, of not more than 
seven
1  other Judges. 
(2) Every Judge of the Supreme Court shall be appo

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

In [15]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [16]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [17]:
response = rag_chain.invoke({"input": "what is citizenship?"})
print(response["answer"])

Articles 5-11 of the Indian Constitution define citizenship.  Article 5 describes citizenship at the commencement of the Constitution, while subsequent articles address specific situations like migration and acquisition of foreign citizenship.  Article 11 empowers Parliament to legislate on matters of citizenship.



In [19]:
response = rag_chain.invoke({"input": "who can vote in india?"})
print(response["answer"])

Indian citizens who are at least 18 years old can vote, unless disqualified by law.  The Constitution specifies qualifications for various elected offices, but voter eligibility is determined by age and citizenship.  Additional voting laws are established by Parliament.



In [20]:
response = rag_chain.invoke({"input": "What is preamble"})
print(response["answer"])

The Preamble to the Constitution of India declares India a sovereign, socialist, secular, democratic republic.  It also states the Constitution's goals of securing justice, liberty, equality, and fraternity to all citizens.  The Preamble was adopted on November 26, 1949, by the Constituent Assembly.



In [21]:
response = rag_chain.invoke({"input": "What do you mean by fraternity"})
print(response["answer"])

Fraternity, in the context of the Indian Constitution, refers to a spirit of brotherhood among all citizens.  It emphasizes the dignity of the individual and the unity and integrity of the nation.  It promotes a sense of common belonging and shared destiny among diverse communities.



In [22]:
response = rag_chain.invoke({"input": "What does article 37 mention?"})
print(response["answer"])

Article 37 states that the provisions of Part IV (Directive Principles of State Policy) are not enforceable by any court. However, these principles are fundamental in governance, and the State has a duty to apply them in making laws.

